# Drew K's final Capstone Assignment - ASDOOR Limited and David's big move to Toronto

### A. Introduction - Outlining problem or idea summary <a name="introduction"></a>

A businessman (David) has been asked by the company that he works for (ASDOOR Limited), to move to Toronto Canada to setup a new warehouse where ASDOOR would like begin operations from, in order to supply the city of Toronto in a more timely manner, and also make use of Toronto's more lenient tax programmes, and cheap rental rates. As part of this move David will need to move his entire family (David and his wife, and 2 children) to the city of Toronto, which will be a significant move for this family. As such David will need to carefully consider where he would like the family to settle in Toronto to ensure he is close to venues and attractions that the family would like to regularly visit. Also, David has a great responsibility for ASDOOR Limited to ensure he chooses a great location for the ASDOOR factory / warehouse to set up operations.

Thus in my assignment I will study the location data available for Toronto Canada, to determine appropriate locations that the ASDOOR factory/warehouse should locate itself, and similarly where David's family should buy/rent a home in order to have an enjoyable living experience in Toronto.

### B. Describe the data that you will be using to solve the problem or execute your idea

I will be using the postal codes of Canada dataset scraped from the web (wikipedia) to determine the boroughs/neighbourhoods of Canada. I will then begin examining the data, stripping down the data to look at Toronto city only. Once I have determined the Toronto postcodes/neighbourhoods I will then analyse;
1. By using Foursquare data, I will determine the number of venues in each neighbourhood
2. Using Foursquare data, I will then determine the unique categories of venues in those neighbourhoods, and determine the top 5 categories of venue in each neighbourhood
3. I will then cluster those neighbourhoods to determine similar cluster types (similar neighbourhoods)
4. Once I have hopefully found distinct clusters, I will investigate each cluster type to determine where is appropriate to setup a warehouse for ASDOOR Limited and where is possibly appropriate for David and his family to settle.
5. Finally, once I believe I have possibly determined the appropriate locations for ASDOOR Limited to develop its warehouse, and Davids family to rent/buy their house, I will specifically investigate those 2 locations for a more detailed analysis of venues so that I am comfortable that the areas are appropriate for the requirements (those being 1. Develop warehouse in appropriate location, and 2. Settle Davids family in a nice postcode that will resonate with the family.

Note: The data that is being scraped is found here: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### C. Methodology to determine core requirements 1. Where should ASDOOR locate its Factory / Warehouse and 2. Where should Davids family choose to live

##### C1. Load Dependencies

In [2]:
#Core dependencies
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [4]:
#Need this patch to enable the remainder dependencies to load correctly. Unsure why was necessary - yet enabled to work / execture
!pip install folium

In [5]:
#Remainder dependencies
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

##### C2: Get the coordinates for Canada neighbourhoods and then drill down further to only look at Toronto neighbourhoods

In [7]:
#Prepare data and determine number of rows
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head(10)
#ignore where borough is not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#use lambda function to join where same postcode exists for cases where different neighbourhood
df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
#where borough exists and neighbourhood does not exist = set nei. == bor.
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']
#notebook clean
print ("no. rows= ",df.shape[0])

no. rows=  103


In [8]:
#Get the coordinates for prepared data
geo_coords=pd.read_csv("https://cocl.us/Geospatial_data")
df['Latitude']=geo_coords['Latitude'].values
df['Longitude']=geo_coords['Longitude'].values
df

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [9]:
#Drill down to Toronto only postcodes / neighbourhoods
#new df only borough, toronto

toro_df = df[ df.Borough.str.contains('Toronto') ]
toro_df.reset_index(inplace=True)
toro_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Rathnelly,Deer Park,Summerhill West,Forest Hil...",43.686412,-79.400049


##### C3: Get Toronto Geographical coordinates and produce map of Toronto with all neighbourhoods (not yet clustered)

In [10]:
#Explore the neighbourhoods in Toronto. 
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada is 43.653963, -79.387207.


In [11]:
# create map of Toronto
map_toro = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toro_df['Latitude'], toro_df['Longitude'], toro_df['Borough'], toro_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toro)  
    
map_toro